# Example 3 - simulation

## Part 1 - MCQ

1. d - SparkStreaming provides Checkpoints to recover from failures;
2. d - The accumulator is used to count the words containing less than 3 letters, while the value 'v' is the sum of the lengths of the words.

## Part 2 - Problems

### 1 - High cpu usage in the working hours of may 2018

In [28]:
!hdfs dfs -rm -r /user/s315054/exam_sim/example3/p01

23/01/26 15:19:38 INFO fs.TrashPolicyDefault: Moved: 'hdfs://BigDataHA/user/s315054/exam_sim/example3/p01' to trash at: hdfs://BigDataHA/user/s315054/.Trash/Current/user/s315054/exam_sim/example3/p01


In [29]:
# RDDs
import sys

# out_file_1 = sys.argv[1]
# in_file = sys.argv[2]

out_file_1 = '/user/s315054/exam_sim/example3/p01'
in_file = '/data/students/bigdata_internet/exam_examples_data/example3_data/PerformanceLog.txt'

in_RDD = sc.textFile(in_file).map(lambda l: l.split(','))\
    .filter(lambda lst: int(lst[0].split('/')[0]) == 2018 and int(lst[0].split('/')[1]) == 5 and (int(lst[1].split(':')[0]) >= 9 and int(lst[1].split(':')[0]) < 18))

crit_RDD = in_RDD.filter(lambda l: float(l[3]) > 99.8).map(lambda el: (el[2], el[3])).mapValues(lambda v1: 1)\
    .reduceByKey(lambda v1, v2: v1+v2).filter(lambda el: int(el[1]) > 10000).keys()

crit_RDD.saveAsTextFile(out_file_1)

In [31]:
in_RDD.collect()

[['2018/05/01', '15:40', 'VS1', '10.5', '0.5'],
 ['2018/05/01', '15:41', 'VS1', '10.5', '0.5'],
 ['2018/05/01', '15:42', 'VS1', '10.5', '0.5'],
 ['2018/05/01', '15:43', 'VS1', '10.5', '0.5'],
 ['2018/05/01', '15:40', 'VS2', '99.9', '100.0'],
 ['2018/05/01', '15:41', 'VS2', '100.0', '0.5'],
 ['2018/05/01', '15:42', 'VS2', '98.5', '0.5'],
 ['2018/05/01', '15:43', 'VS2', '90.5', '0.5'],
 ['2018/05/01', '09:43', 'VS3', '90.4', '1.5'],
 ['2018/05/01', '10:43', 'VS3', '90.4', '1.5'],
 ['2018/05/01', '11:43', 'VS3', '90.4', '1.5'],
 ['2018/05/01', '12:43', 'VS3', '9.4', '1.5'],
 ['2018/05/01', '13:43', 'VS3', '90.4', '1.5'],
 ['2018/05/01', '14:43', 'VS3', '9.4', '1.5'],
 ['2018/05/01', '15:43', 'VS3', '8.4', '1.5'],
 ['2018/05/01', '16:43', 'VS3', '90.4', '1.5'],
 ['2018/05/01', '17:43', 'VS3', '90.4', '1.5']]

In [34]:
# DFs
import sys

# out_file_1 = sys.argv[1]
# in_file = sys.argv[2]

out_file_1 = '/user/s315054/exam_sim/example3/p01'
in_file = '/data/students/bigdata_internet/exam_examples_data/example3_data/PerformanceLog.txt'

spark.udf.register('getYear', lambda date: int(date.split('/')[0]))
spark.udf.register('getMonth', lambda date: int(date.split('/')[1]))
spark.udf.register('getHour', lambda hour: int(hour.split(':')[0]))


in_DF = spark.read.load(in_file, format='csv', sep=',', header=False, inferSchema=True)\
    .withColumnRenamed('_c0', 'date')\
    .withColumnRenamed('_c1', 'hour')\
    .withColumnRenamed('_c2', 'vsid')\
    .withColumnRenamed('_c3', 'cpu')\
    .withColumnRenamed('_c4', 'ram')

inter_DF = in_DF.filter("getYear(date) == 2018 AND getMonth(date) == 5 AND getHour(hour) >= 9 AND getHour(hour) < 18")

inter_DF.createOrReplaceTempView('table_1_1')

out_DF = spark.sql("""
                SELECT vsid
                FROM table_1_1
                WHERE cpu > 99.8
                """).groupBy('vsid').agg({'*': 'count'}).filter('count(1)>10000').select('vsid')



23/01/26 15:21:36 WARN analysis.SimpleFunctionRegistry: The function getyear replaced a previously registered function.
23/01/26 15:21:36 WARN analysis.SimpleFunctionRegistry: The function getmonth replaced a previously registered function.
23/01/26 15:21:36 WARN analysis.SimpleFunctionRegistry: The function gethour replaced a previously registered function.


In [33]:
out_DF.show()

+----+--------+
|vsid|count(1)|
+----+--------+
+----+--------+



### 2a - Critical VSID, hour pairs, may 2018

In [ ]:
!hdfs dfs -rm -r /user/s315054/exam_sim/example3/p02a

In [28]:
import sys

# CPUthr = sys.argv[1]
# RAMthr = sys.argv[2]

CPUthr = 50.
RAMthr = 20.

in_file = '/data/students/bigdata_internet/exam_examples_data/example3_data/PerformanceLog.txt'
out_file_2a = '/user/s315054/exam_sim/example3/p02a'

in_RDD = sc.textFile(in_file).map(lambda line: line.split(','))    # Initial RDD
filter_RDD = in_RDD.filter(lambda l: int(l[0].split('/')[0]) == 2018 and int(l[0].split('/')[1]) == 5)\
        .map(lambda l: ((l[2], int(l[1].split(':')[0])), (float(l[3]), float(l[4]), 1) ))      # Keys: (VSID, hour), value (CPUusage, RAMusage, 1)

red_RDD = filter_RDD.reduceByKey(lambda v1, v2: (v1[0] + v2[0], v1[1] + v2[1], v1[2] + v2[2])).mapValues(lambda v: (v[0]/v[2], v[1]/v[2]))

final_RDD = red_RDD.filter(lambda el: el[1][0] > CPUthr and el[1][1] > RAMthr)

## Need to just keep keys
out_RDD = final_RDD.keys().map(lambda el: f"{el[0]}_{el[1]}")
out_RDD.saveAsTextFile(out_file_2a)

In [27]:
final_RDD.collect()

[(('VS2', 15), (97.225, 25.375))]

###  2b - Daily unbalanced usage of the CPU, May 2018

In [17]:
!hdfs dfs -rm -r /user/s315054/exam_sim/example3/p02b

23/01/29 08:52:35 INFO fs.TrashPolicyDefault: Moved: 'hdfs://BigDataHA/user/s315054/exam_sim/example3/p02b' to trash at: hdfs://BigDataHA/user/s315054/.Trash/Current/user/s315054/exam_sim/example3/p02b1674982355294


In [18]:
out_file_2b = '/user/s315054/exam_sim/example3/p02b'

# Start from in_RDD, create pairs (VSID, date) and count the number of max. cpu > 90 and n. of max CPU < 10

# First, isolate elements in May 2018, then map them to key-value pair having as key (VSID, date, hour), before finding the max for each hour
f1_RDD = in_RDD.filter(lambda l: int(l[0].split('/')[0]) == 2018 and int(l[0].split('/')[1]) == 5)\
        .map(lambda l: (( l[2], l[0], int(l[1].split(':')[0]) ), float(l[3]) ))
###################### VSID^, date^,        hour^,                CPU%^

# Find max for each hour:
f2_RDD = f1_RDD.reduceByKey(lambda v1, v2: max(v1, v2))

# Remap to get keys of the type (vsid, date) and map these elements' values to (1 if >90, 1 if <90)
f3_RDD = f2_RDD.map(lambda e: ((e[0][0], e[0][1]), (int(e[1] > 90), int(e[1] < 10)) ))

# Reduce by key to sum corresponding values in the value tuples, then filter
final_RDD = f3_RDD.reduceByKey(lambda v1, v2: (v1[0] + v2[0], v1[1] + v2[1])).filter(lambda el: el[1][0]>=8 and el[1][1] >= 8).keys()

# Produce the required output
output_RDD = final_RDD.map(lambda k: f"{k[0]}_{k[1].replace('/', '-')}")
output_RDD.saveAsTextFile(out_file_2b)


In [19]:
output_RDD.collect()

['VS3_2018-05-01']

#### Using DFs

In [30]:
import sys

in_file = '/data/students/bigdata_internet/exam_examples_data/example3_data/PerformanceLog.txt'

####### a
# CPU_thresh = sys.argv[1]
# RAM_thresh = sys.argv[2]
CPU_thresh = 50.
RAM_thresh = 20.

spark.udf.register('getYear', lambda el: int(el.split('/')[0]))
spark.udf.register('getMonth', lambda el: int(el.split('/')[1]))
spark.udf.register('getHour', lambda el: el.split(':')[0])

in_DF = spark.read.load(in_file, format='csv', sep=',', header=False)\
        .withColumnRenamed('_c0', 'date').withColumnRenamed('_c1', 'time')\
        .withColumnRenamed('_c2', 'vsID').withColumnRenamed('_c3', 'CPU')\
        .withColumnRenamed('_c4', 'RAM')

may_DF = in_DF.filter("getYear(date) == 2018 AND getMonth(date) == 5")

start_DF = may_DF.selectExpr('vsID', 'getHour(time) AS hour', "CPU", "RAM").groupBy("vsID", "hour").agg({"CPU":"avg", "RAM": "avg"})\
                .withColumnRenamed('avg(CPU)', 'avg_CPU').withColumnRenamed('avg(RAM)', "avg_RAM")\
                .createOrReplaceTempView('may_2018')

out_DF = spark.sql(f" \
                    SELECT vsID, hour\
                    FROM may_2018\
                    WHERE avg_CPU > {CPU_thresh} AND avg_RAM > {RAM_thresh}\
                    ")

# Since the header is not needed, can move to RDDs
out_DF2RDD = out_DF.rdd.map(lambda el: f"{el[0]}_{el[1]}")
#out_DF2RDD.saveAsTextFile(out_file_2a)

23/01/29 09:43:37 WARN analysis.SimpleFunctionRegistry: The function getyear replaced a previously registered function.
23/01/29 09:43:37 WARN analysis.SimpleFunctionRegistry: The function getmonth replaced a previously registered function.
23/01/29 09:43:37 WARN analysis.SimpleFunctionRegistry: The function gethour replaced a previously registered function.


In [ ]:
!hdfs dfs -rm -r /user/s315054/exam_sim/example3/p02b_df

In [40]:
# Part 2b

out_file_2b_df = '/user/s315054/exam_sim/example3/p02b_df'

# Start from may_DF
# Need to evaluate the maximum value of CPU usage in each hour
# Columns: date, time, vsID, CPU, RAM

sel_DF = may_DF.selectExpr('vsID', 'date', 'getHour(time) AS hour', 'CPU')\
        .groupBy('vsID', 'date', 'hour').agg({'CPU':'max'})\
        .withColumnRenamed('max(CPU)', 'max_CPU')

sel_DF.createOrReplaceTempView('selected')

count_DF = spark.sql("\
            SELECT vsID, date\
            FROM selected\
            GROUP BY vsID, date\
            HAVING count(max_CPU > 90) >= 8 AND count(max_CPU < 10) >= 8\
            ")

count_DF.write.csv(out_file_2b_df, header=False, sep='_')

In [45]:
in_file = '/data/students/bigdata_internet/exam_examples_data/example3_data/PerformanceLog.txt'

in_DF = spark.read.load(in_file, format='csv', sep=',', header=False).toDF('date', 'time', 'vsID', 'CPU', 'RAM')
in_DF.show(5)

+----------+-----+----+----+---+
|      date| time|vsID| CPU|RAM|
+----------+-----+----+----+---+
|2018/05/01|15:40| VS1|10.5|0.5|
|2018/05/01|15:41| VS1|10.5|0.5|
|2018/05/01|15:42| VS1|10.5|0.5|
|2018/05/01|15:43| VS1|10.5|0.5|
|2018/05/01|05:40| VS1|10.5|0.5|
+----------+-----+----+----+---+
only showing top 5 rows

